# PDF 문서를 image로 변환 후 요약하는 샘플 코드

In [1]:
!pip install PyMuPDF Pillow boto3

## 1. 매서드 작성

In [11]:
import fitz  # PyMuPDF
import boto3
import base64
from PIL import Image
import io
import json

In [12]:
# 1. PDF를 이미지로 변환
def pdf_to_images(pdf_path):
    doc = fitz.open(pdf_path)
    images = []
    for page in doc:
        pix = page.get_pixmap()
        img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
        images.append(img)
    return images

In [13]:
# 2. 이미지를 Base64로 인코딩
def image_to_base64(image):
    buffered = io.BytesIO()
    image.save(buffered, format="PNG")
    return base64.b64encode(buffered.getvalue()).decode()

In [14]:
# 3. Amazon Bedrock 클라이언트 설정
bedrock = boto3.client(
    service_name='bedrock-runtime',
    region_name='us-east-1'  # 예: 'us-west-2'
)

In [15]:
# 4. Claude 모델에 이미지 전송 및 응답 받기
def get_claude_response(image_base64):
    prompt = "소제목 별로 요약을 해주세요."
    body = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 1000,
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "image",
                        "source": {
                            "type": "base64",
                            "media_type": "image/png",
                            "data": image_base64
                        }
                    },
                    {
                        "type": "text",
                        "text": prompt
                    }
                ]
            }
        ]
    }

    #아래 modelId 는 Claude 3.5 Sonnet 으로 변경하세요.
    response = bedrock.invoke_model(
        body=json.dumps(body),
        #modelId = "anthropic.claude-3-5-sonnet-20240620-v1:0"  # Claude 3.5 Sonnet
        modelId="anthropic.claude-3-sonnet-20240229-v1:0",  # Claude 3 Sonnet
        contentType="application/json",
        accept="application/json"
    )

    response_body = json.loads(response['body'].read())
    return response_body['content'][0]['text']

## 2. Claude 3.x 모델 호출하여 이미지 pdf 처리

In [16]:
pdf_path = "./rag_data/Steel Business Briefing_04 Jul 2024.pdf"
images = pdf_to_images(pdf_path)

for i, image in enumerate(images):
    image_base64 = image_to_base64(image)
    response = get_claude_response(image_base64)
    print(f"Page {i+1} Summary:")
    print(response)
    print("\n---\n")

Page 1 Summary:
물론이죠. 이미지의 각 섹션별로 중요한 내용을 요약해드리겠습니다.

Price snapshot:
여러 원자재의 일일 및 주간 가격 정보를 표로 제공하고 있습니다.

Contents:
이 문서의 주요 내용 섹션들을 열거하고 있습니다. 아시아, 아메리카, 유럽/중동/아프리카 지역별 원자재 시황과 업데이트 사항, 그리고 구독자를 위한 참고사항들을 다룹니다.

Asia:
아시아 지역의 원유, 석유제품, 나프타 등 시황 정보를 제공합니다.

Americas: 
미 석유제품 수요 전망과 브라질 연료가격 인상 소식 등을 포함하고 있습니다.

EMEA:
유럽, 중동, 아프리카 지역의 원유, 석유제품, LPG 등 시황과 주요 이슈를 다룹니다. 

CIS:
구소련 지역 원자재 관련 정보를 제공하고 있습니다.

이처럼 이 문서는 전 세계 주요 원자재 시황과 관련 뉴스를 지역별로 집약하여 제공하고 있습니다.

---

Page 2 Summary:
물론입니다. 이 이미지에 포함된 주요 섹션들을 요약해드리겠습니다.

Asia:
중국의 철광석 가격은 계속 약세를 보였고, 인도 시장과 베트남 시장도 거래가 부진했습니다. 철광석 스왑 가격은 하락했지만, 중국발 구매 inquiries는 거의 없었습니다. 아시아 철광석 시장은 전반적으로 구매력이 약했습니다.

Flat Products: 
아시아 HRC의 수요와 구매 관심이 낮았고, 딜러들의 재고 적체 현상도 있었습니다. 일본과 한국의 수출 오퍼는 약 $10/mt 수준이었습니다.

Indian rebar prices dip as mills lower offers to ignite market HRC stable:
인도 철강사들이 판매를 촉진하기 위해 리바 가격을 내렸지만, HRC 가격은 안정세를 보였습니다.

요약하자면 아시아 철강 시장이 전반적으로 수요 부진과 약세를 보이고 있음을 알려주는 내용입니다.

---

Page 3 Summary:
물론입니다. 각 섹션 별로 요약해 드리겠습니다.

Long Products